# Take eddy that collocated to CPR data and find complete eddy history
- Read in collocated bird data
- Read in AVISO eddy database subset for NPac
- find unique eddies in bird data
- find all AVISO eddy for that eddy ID
- create new dataset with track number and observation number as dims
- save eddy data into it

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import warnings
# filter some warning messages
warnings.filterwarnings("ignore") 
from geopy.distance import geodesic 
from pyresample.kd_tree import resample_nearest
from scipy import spatial

####################you will need to change some paths here!#####################
#list of input files
#list of input files

filename_aviso='F:/data/project_data/NASA_biophysical/aviso/eddy_trajectory_19930101_20170106.nc'   #From AVISO  website
filename_bird_netcdf='f:/data/project_data/NASA_biophysical/collocated_data/CPR/All CPR Sample catalogue with eddy info_2020_10_06.nc'
#################################################################################
filename_origin='f:/data/project_data/NASA_biophysical/collocated_data/CPR/All_CPR_Sample_catalogue_with_eddy_info_2020_10_06_origin_file.nc'

# read in collocated bird data

In [ ]:
ds_bird = xr.open_dataset(filename_bird_netcdf)
ds_bird

# open aviso eddy dataset

In [2]:
ds_eddy = xr.open_dataset('F:/data/project_data/NASA_biophysical/aviso/eddy_trajectory_19930101_20170106.nc')
ds_eddy = ds_eddy.rename({'latitude':'lat','longitude':'lon'})
#ds_eddy = xr.open_dataset('F:/data/project_data/NASA_biophysical/aviso/eddy_trajectory_19930101_20170106_northpac.nc')
ilen_eddy = len(ds_eddy.lat)
ds_eddy

<xarray.Dataset>
Dimensions:             (obs: 23086878)
Dimensions without coordinates: obs
Data variables:
    amplitude           (obs) float32 ...
    cyclonic_type       (obs) int8 ...
    lat                 (obs) float32 ...
    lon                 (obs) float32 ...
    observation_number  (obs) uint16 ...
    speed_average       (obs) float32 ...
    speed_radius        (obs) float32 ...
    time                (obs) object ...
    track               (obs) uint32 ...
Attributes: (12/16)
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    title:                     Mesoscale Eddies in Altimeter Observations of SLA
    institution:               CNES/CLS in collaboration with Oregon State Un...
    project:                   SSALTO/DUACS
    creator_url:               http://www.aviso.altimetry.fr
    creator_email:             aviso@altimetry.fr
    ...                        ...
    date_created:              2017-09-22T10:20:17Z
    time_coverage_duration:    P8771D
    time_coverage_start:       1993-01-01T00:00:00Z
    time_coverage_end:         2017-01-06T00:00:00Z
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    product_version:           1.0

In [ ]:
ds_eddy['time64']=xr.DataArray(np.empty(len(ds_eddy.lat),dtype='datetime64[ns]'), dims=('obs'))
for i,time in enumerate(ds_eddy.time.data):
    ds_eddy['time64'][i]=np.datetime64(time)

In [ ]:
ds_eddy

# create a array where True when any of eddy database matches with bird collocated eddy

In [ ]:
ds_bird

In [ ]:
ds2

In [ ]:
%%time
ds2 = ds_bird.where(ds_bird.cpr_eddy_data_track>1,drop=True)
all_tracks = np.unique(ds2.cpr_eddy_data_track.data)
result = ds_eddy.amplitude*np.nan
#result = [x for x in a if x in b]
a = all_tracks
b = ds_eddy.track.astype('int')
for val in a:
    result = xr.where(b==val,True,result) #,1,b) #,True,result)
print(result.sum())
ds_eddy = ds_eddy.where(result==True,drop=True)
ds_eddy

In [ ]:
plt.scatter(ds_eddy.lon,ds_eddy.lat)

# Create xarray dataset structure to hold data

In [ ]:
len(all_tracks)

In [ ]:
ds2 = ds_bird.where(ds_bird.cpr_eddy_data_track>1,drop=True)
all_tracks = np.unique(ds2.cpr_eddy_data_track.data)
ilen = len(all_tracks)
inum_obs = int(ds_eddy.observation_number.max().data)+1
iobs = np.arange(inum_obs)
print(ilen,inum_obs)
da=xr.DataArray(np.nan*np.empty((ilen,inum_obs), 
                                dtype=str(ds_eddy['amplitude'].dtype)), 
                coords={'track': all_tracks.astype('int'),'observation_number':np.arange(inum_obs)}, 
                dims=('track','observation_number'))
ds_tem = xr.Dataset({'amplitude':da})
for var in ds_eddy:
    if (var!='observation_number') & (var!='track') & (var!='time') & (var!='time64') & (var!='amplitude'):
        da=xr.DataArray(np.nan*np.empty((ilen,inum_obs), 
                                dtype=str(ds_eddy[var].dtype)), 
                coords={'track': all_tracks.astype('int'),'observation_number':np.arange(inum_obs)}, 
                dims=('track','observation_number'))

        ds_tem[var]=da
    if (var=='time64'):
        da=xr.DataArray(np.empty((ilen,inum_obs), 
                                dtype=str(ds_eddy[var].dtype)), 
                coords={'track': all_tracks.astype('int'),'observation_number':np.arange(inum_obs)}, 
                dims=('track','observation_number'))

        ds_tem[var]=da
ds_tem

# go through all unique eddies and save data into structure
- save variable attributes
- save global attributes

In [ ]:
for ii,idn in enumerate(ds_tem.track.data):
    subset = ds_eddy.where(ds_eddy.track.astype('int')==idn,drop=True)
    for var in subset:
        if (var!='observation_number') & (var!='track') & (var!='time') & (var!='amplitude'):
            ilen_obs = len(subset.obs)
            ds_tem[var][ii,:ilen_obs]=subset[var][:].data
            ds_tem[var].attrs=subset[var].attrs
    if subset.observation_number[0].data>0:
        print(ii,subset.observation_number[0])
ds_tem.attrs = ds_eddy.attrs
ds_tem.to_netcdf(filename_origin)

In [ ]:
ds_tem